![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

# Spark NLP Quick Start
### How to use Spark NLP pretrained pipelines

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/quick_start_google_colab.ipynb)

We will first set up the runtime environment and then load pretrained Entity Recognition model and Sentiment analysis model and give it a quick test. Feel free to test the models on your own sentences / datasets.

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-03-01 04:07:51--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-03-01 04:07:52--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-03-01 04:07:52 (50.5 MB/s) - written to stdout [1191/1191]

Installing

In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.3.0
Apache Spark version: 3.2.3


In [ ]:
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
pipeline = PretrainedPipeline('analyze_sentimentdl_glove_imdb', 'en')

analyze_sentimentdl_glove_imdb download started this may take some time.
Approx size to download 154.1 MB
[OK!]


In [ ]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.3 MB/s eta 0:00:00


In [ ]:
import praw
import pandas as pd
import re

In [ ]:
keyword = 'comedy'

In [ ]:
# Initialize PRAW with Reddit API credentials
reddit = praw.Reddit(client_id='R2hyuU2V_wnLGBZyFEVLtw',
                     client_secret='ayv3CTLaTdIot3qpXqqoGCrQq8KM3A',
                     user_agent='web bot')

subreddit = reddit.subreddit('shortstories')

# Collect flairs/tags
flair_counts = {}
for post in subreddit.new(limit=100):
    flair = post.link_flair_text
    if flair:
        flair_counts[flair] = flair_counts.get(flair, 0) + 1

flair_df = pd.DataFrame(list(flair_counts.items()), columns=['Flair', 'Count'])
print(flair_df.sort_values(by='Count', ascending=False))

def scrape_posts_by_keyword(subreddit_name, keyword, limit=50):
    subreddit = reddit.subreddit(subreddit_name)
    posts_data = []

    for post in subreddit.search(keyword, limit=limit):
        post_data = {
            'title': post.title,
            'author': str(post.author),
            'score': post.score,
            'id': post.id,
            'url': post.url,
            'created_utc': post.created_utc,
            'selftext': post.selftext,
            'num_comments':post.num_comments
        }
        posts_data.append(post_data)

    return pd.DataFrame(posts_data)
# keyword = 'Emotional'
posts_df = scrape_posts_by_keyword('shortstories', keyword)
posts_df['created_utc'] = pd.to_datetime(posts_df['created_utc'], unit='s')
#pre processing the self text for ml
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove anything that is not a letter, number, punctuation, or whitespace
    text = re.sub(r'[^\w\s\.,!?]', '', text)
    return text

posts_df['selftext'] = posts_df['selftext'].apply(clean_text)


posts_df.head()

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



                  Flair  Count
1       Science Fiction     26
0               Fantasy     18
2          Misc Fiction     11
3     Realistic Fiction     11
8                Horror     10
10               Humour      4
4   Speculative Fiction      3
9              Thriller      3
11        Serial Sunday      3
5    Mystery & Suspense      2
6               Romance      2
7             Off Topic      2
14         Micro Monday      2
12   Historical Fiction      1
13            Meta Post      1
15   Action & Adventure      1


,title,author,score,id,url,created_utc,selftext,num_comments
0,[OT] Micro Monday: Pumpkin Surprise / Comedy H...,OldBayJ,9,xuskl0,https://www.reddit.com/r/shortstories/comments...,2022-10-03 19:15:25,Welcome to Micro Monday\n\nHello writers and w...,44
1,"[SF] I Can’t Stop Watching Four-Legged Comedy,...",RobertSage,3,13sf2sd,https://www.reddit.com/r/shortstories/comments...,2023-05-26 14:42:31,"I Cant Stop Watching FourLegged Comedy, And It...",1
2,[SF/Comedy] Technology Sucks (Extract),Forest_ftm,6,m49c3e,https://www.reddit.com/r/shortstories/comments...,2021-03-13 16:12:04,"It started a few weeks ago, and honestly, I th...",0
3,[HM][SF] Kill'em With Comedy,patrickryan182,15,fvv4a1,https://www.reddit.com/r/shortstories/comments...,2020-04-06 08:36:03,Logline In a futuristic world where society is...,4
4,[NF][Comedy] The elderly gentleman wishes to die.,AnasQiblawi,2,n1b7fr,https://www.reddit.com/r/shortstories/comments...,2021-04-29 18:47:22,There was once a man who was so old that he ap...,1


In [ ]:
# Initialize a Spark session
from pyspark.sql import SparkSession
sparksession = SparkSession.builder \
    .appName("Reddit Data Processing") \
    .getOrCreate()

In [ ]:
spark_df = sparksession.createDataFrame(posts_df)

spark_df.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+
|               title|              author|score|     id|                 url|        created_utc|            selftext|num_comments|
+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+
|[MS] Who's really...|      Inevitable_Jex|    6| r463ol|https://www.reddi...|2021-11-28 14:58:38|This is how it st...|           1|
|[MF] The Peak of ...|            gammarik|    3| nwvq2z|https://www.reddi...|2021-06-10 19:08:57|The mother climbs...|           4|
|[RO] Logic and Em...|          Sensorfire|    5| 2o08mn|https://www.reddi...|2014-12-02 02:43:15|This is my first ...|           4|
|[MT] Need help fi...|             Skrot44|    0| 67ib3e|https://www.reddi...|2017-04-25 18:05:00|Hello all, Im her...|           0|
|[RO] My short sto...|      MoneyTakerBaby|    6|11kzcuc|https://www.

In [ ]:

# Verify the DataFrame structure (Optional but helpful)
spark_df.printSchema()


root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: long (nullable = true)
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- selftext: string (nullable = true)
 |-- num_comments: long (nullable = true)



In [ ]:
# Ensure your DataFrame has a column named "text" for the pipeline
# Rename the 'selftext' column to 'text'
prepared_df = spark_df.withColumnRenamed("selftext", "text")


In [ ]:

# Now, apply the sentiment analysis pipeline to the correctly prepared DataFrame
result_df = pipeline.transform(prepared_df)


In [ ]:

# Show the original text and the sentiment result
# Assuming the pipeline outputs a column named 'sentiment.result'
# Adjust the column names based on the actual output of your pipeline
result_df.select("title", "sentiment.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|title                                                                                                                                                                                                                                                               |result   |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|[MS] Who's really to blame? (Emotions or The Mission?                                                                                                                               

In [ ]:
pipeline2 = PretrainedPipeline("analyze_sentimentdl_use_imdb", lang = "en")

analyze_sentimentdl_use_imdb download started this may take some time.
Approx size to download 935.7 MB
[OK!]


In [ ]:
result_df2 = pipeline2.transform(prepared_df)


In [ ]:

result_df2.select("title", "sentiment.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|title                                                                                                                                                                                                                                                               |result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|[MS] Who's really to blame? (Emotions or The Mission?                                                                                                                                        

In [ ]:
from pyspark.sql.functions import col, expr

# Extract sentiment and sentiment score from the first pipeline's results
test1 = result_df.withColumn("sentiment1", expr("sentiment.result[0]"))
# Assuming a way to calculate sentiment score1, as Spark NLP's output format varies

# Extract sentiment and sentiment score from the second pipeline's results
test2 = result_df2.withColumn("sentiment2", expr("sentiment.result[0]"))
# Assuming a way to calculate sentiment score2


In [ ]:
test1.show()

+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|              author|score|     id|                 url|        created_utc|                text|num_comments|            document|            sentence|              tokens|     word_embeddings| sentence_embeddings|           sentiment|sentiment1|
+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|[MS] Who's really...|      Inevitable_Jex|    6| r463ol|https://www.reddi...|2021-11-28 14:58:38|This is how it st...|           1|[{document, 0, 11...|[{document, 0, 21...|[{token, 0, 3, T

In [ ]:
test2.show()

+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------------------+----------+
|               title|              author|score|     id|                 url|        created_utc|                text|num_comments|            document| sentence_embeddings|           sentiment|sentiment2|
+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------------------+----------+
|[MS] Who's really...|      Inevitable_Jex|    6| r463ol|https://www.reddi...|2021-11-28 14:58:38|This is how it st...|           1|[{document, 0, 11...|[{sentence_embedd...|[{category, 0, 11...|       neg|
|[MF] The Peak of ...|            gammarik|    3| nwvq2z|https://www.reddi...|2021-06-10 19:08:57|The mother climbs...|           4|[{document, 0, 51...|[{sentence_embedd..

In [ ]:

test1.select("title", "sentiment").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+
|title                                                                                                                                                                                                                                                               |sentiment                                                                                |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------

In [ ]:
# Join the original DataFrame with the first pipeline's results
combined_df = spark_df.join(test1.select("title", "sentiment1"), "title")

# Join the combined DataFrame with the second pipeline's results
final_df = combined_df.join(test2.select("title", "sentiment2"), "title")


In [ ]:
final_df.show()

+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+----------+----------+
|               title|              author|score|     id|                 url|        created_utc|            selftext|num_comments|sentiment1|sentiment2|
+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+----------+----------+
|      [SF] EMO-DRIVE|        Daplugottawa|    2|11unsno|https://www.reddi...|2023-03-18 12:56:42|Title EmoDrive Th...|           1|       pos|       pos|
|[OT] Micro Monday...|             OldBayJ|    7|18as6of|https://www.reddi...|2023-12-04 19:35:09|Welcome to Micro ...|          17|       pos|       pos|
|[OT] Micro Monday...|             OldBayJ|   14|12pvw78|https://www.reddi...|2023-04-17 20:38:12|Welcome to Micro ...|          32|       pos|       pos|
|[FN] Path of Chan...|             Aftel43|    2|18hjv4t|https://www.r

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import lit


In [ ]:
# def analyze_keyword(keywordforscraping):
#     # Scrape posts
#     posts_df = scrape_posts_by_keyword('shortstories', keywordforscraping)
#     posts_df['created_utc'] = pd.to_datetime(posts_df['created_utc'], unit='s')

#     def clean_text(text):# Remove URLs

#       text = re.sub(r'http\S+', '', text)
#       # Remove anything that is not a letter, number, punctuation, or whitespace
#       text = re.sub(r'[^\w\s\.,!?]', '', text)
#       return text

#     posts_df['selftext'] = posts_df['selftext'].apply(clean_text)


#     # Convert to Spark DataFrame
#     spark_df = spark.createDataFrame(posts_df)

#     # # Preprocess text
#     # spark_df = spark_df.withColumn("clean_text", clean_text_udf(col("selftext")))

#     # Rename column for sentiment analysis
#     prepared_df = spark_df.withColumnRenamed("selftext", "text")

#     # Analyze sentiment with both pipelines
#     result_df1 = pipeline.transform(prepared_df)
#     result_df2 = pipeline2.transform(prepared_df)

#       # Extract sentiment and sentiment score from the first pipeline's results
#     test1 = result_df.withColumn("sentiment1", expr("sentiment.result[0]"))
#     # Assuming a way to calculate sentiment score1, as Spark NLP's output format varies

#     # Extract sentiment and sentiment score from the second pipeline's results
#     test2 = result_df2.withColumn("sentiment2", expr("sentiment.result[0]"))
#     # Assuming a way to calculate sentiment score2
#     # Join the original DataFrame with the first pipeline's results
#     combined_df = spark_df.join(test1.select("title", "sentiment1"), "title")

#     # Join the combined DataFrame with the second pipeline's results
#     final_df = combined_df.join(test2.select("title", "sentiment2"), "title")
#     final_df = final_df.withColumn("keyword", lit(keywordforscraping))
#     # Your existing code to scrape, process, and analyze sentiment...
#     print(f"Processing keyword: {keywordforscraping}")
#     # Assuming the final DataFrame for this keyword is named `result_df`
#     print(f"Number of posts for {keywordforscraping}: {final_df.count()}")

#     final_df.show(5)
#     return final_df


In [ ]:
def analyze_keyword(keywordforscraping):
    print(f"Starting processing for keyword: {keywordforscraping}")
    try:
        # Your existing code to scrape, process, and analyze sentiment...
        # For example:
        def scrape_posts_by_keyword(subreddit_name, keyword, limit=50):
            subreddit = reddit.subreddit(subreddit_name)
            posts_data = []

            for post in subreddit.search(keyword, limit=limit):
                post_data = {
                    'title': post.title,
                    'author': str(post.author),
                    'score': post.score,
                    'id': post.id,
                    'url': post.url,
                    'created_utc': post.created_utc,
                    'selftext': post.selftext,
                    'num_comments':post.num_comments
                }
                posts_data.append(post_data)

            return pd.DataFrame(posts_data)
        # posts_df = scrape_posts_by_keyword('shortstories', keyword)
        posts_df = scrape_posts_by_keyword('shortstories', keywordforscraping)
        if posts_df.empty:
            print(f"No posts found for keyword: {keywordforscraping}")
            return None  # Return None or an empty DataFrame
        posts_df['created_utc'] = pd.to_datetime(posts_df['created_utc'], unit='s')

        def clean_text(text):# Remove URLs

          text = re.sub(r'http\S+', '', text)
          # Remove anything that is not a letter, number, punctuation, or whitespace
          text = re.sub(r'[^\w\s\.,!?]', '', text)
          return text

        posts_df['selftext'] = posts_df['selftext'].apply(clean_text)

        # Convert to Spark DataFrame, preprocess, analyze sentiment, etc.
        # Example:
        spark_df = spark.createDataFrame(posts_df)
        print(f"Converted to Spark DataFrame for keyword: {keywordforscraping}, Number of posts: {spark_df.count()}")

        # Assume you have the rest of your pipeline here
        # Rename column for sentiment analysis
        prepared_df = spark_df.withColumnRenamed("selftext", "text")

        # Analyze sentiment with both pipelines
        result_df1 = pipeline.transform(prepared_df)
        result_df2 = pipeline2.transform(prepared_df)

          # Extract sentiment and sentiment score from the first pipeline's results
        test1 = result_df1.withColumn("sentiment1", expr("sentiment.result[0]"))
        # Assuming a way to calculate sentiment score1, as Spark NLP's output format varies

        # Extract sentiment and sentiment score from the second pipeline's results
        test2 = result_df2.withColumn("sentiment2", expr("sentiment.result[0]"))
        # Assuming a way to calculate sentiment score2
        # Join the original DataFrame with the first pipeline's results
        combined_df = spark_df.join(test1.select("title", "sentiment1"), "title")

        # Join the combined DataFrame with the second pipeline's results
        final_df = combined_df.join(test2.select("title", "sentiment2"), "title")
        final_df = final_df.withColumn("keyword", lit(keywordforscraping))
        # Your existing code to scrape, process, and analyze sentiment...
        print(f"Processing keyword: {keywordforscraping}")
        # Assuming the final DataFrame for this keyword is named `result_df`
        print(f"Number of posts for {keywordforscraping}: {final_df.count()}")

        final_df.show(5)
        print(f"Completed processing for keyword: {keywordforscraping}")
        return final_df  # Or your final DataFrame for this keyword

    except Exception as e:
        print(f"Error processing keyword {keywordforscraping}: {e}")
        return None  # Return None or an empty DataFrame on error

In [ ]:
keywords = ["Emotional", "adventure", "mystery","horror","comedy"]
dfs = [analyze_keyword(key) for key in keywords]



It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Starting processing for keyword: Emotional


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Converted to Spark DataFrame for keyword: Emotional, Number of posts: 50
Processing keyword: Emotional
Number of posts for Emotional: 50


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

Converted to Spark DataFrame for keyword: horror, Number of posts: 50
Processing keyword: horror
Number of posts for horror: 50


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



+--------------------+------------------+-----+------+--------------------+-------------------+--------------------+------------+----------+----------+-------+
|               title|            author|score|    id|                 url|        created_utc|            selftext|num_comments|sentiment1|sentiment2|keyword|
+--------------------+------------------+-----+------+--------------------+-------------------+--------------------+------------+----------+----------+-------+
|[HR] Museum Horro...|   hhhhiiiiiiiiiii|   10|qe8w5x|https://www.reddi...|2021-10-23 17:07:05|I was alone in th...|           3|       neg|       neg| horror|
|[HR] Treehouse of...|  BigNickOnTheDrum|    2|q5bc0p|https://www.reddi...|2021-10-10 16:27:01|Bart is upset tha...|           3|       pos|   neutral| horror|
|[HR] Horror/Fanta...|clayvermulmfiction|   13|mqr3li|https://www.reddi...|2021-04-14 13:56:43|HR Seismic Part 1...|           4|       pos|       neg| horror|
|[OT] Roundtable T...|           OldBayJ

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Converted to Spark DataFrame for keyword: comedy, Number of posts: 50
Processing keyword: comedy
Number of posts for comedy: 50
+--------------------+------------------+-----+------+--------------------+-------------------+--------------------+------------+----------+----------+-------+
|               title|            author|score|    id|                 url|        created_utc|            selftext|num_comments|sentiment1|sentiment2|keyword|
+--------------------+------------------+-----+------+--------------------+-------------------+--------------------+------------+----------+----------+-------+
|[RF] The Human Co...|          vladbrat|    2|2n0cdh|http://www.imagec...|2014-11-21 19:00:38|                    |           0|       neg|      null| comedy|
|[RF] The Human Co...|          xidanphi|    1|2sepj7|http://www.imagec...|2015-01-14 15:39:14|                    |           0|       neg|       pos| comedy|
|[RF] The Human Co...|          xidanphi|    3|2nhgev|http://www.imagec.

In [ ]:

# Merge all DataFrames into one
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

combine_final_df = unionAll(*dfs)


In [ ]:
dfs[0]

DataFrame[title: string, author: string, score: bigint, id: string, url: string, created_utc: timestamp, selftext: string, num_comments: bigint, sentiment1: string, sentiment2: string, keyword: string]

In [ ]:
combine_final_df.show()

+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+----------+----------+---------+
|               title|              author|score|     id|                 url|        created_utc|            selftext|num_comments|sentiment1|sentiment2|  keyword|
+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+------------+----------+----------+---------+
|      [SF] EMO-DRIVE|        Daplugottawa|    2|11unsno|https://www.reddi...|2023-03-18 12:56:42|Title EmoDrive Th...|           1|       pos|       pos|Emotional|
|[OT] Micro Monday...|             OldBayJ|    5|18as6of|https://www.reddi...|2023-12-04 19:35:09|Welcome to Micro ...|          17|       pos|       pos|Emotional|
|[OT] Micro Monday...|             OldBayJ|   14|12pvw78|https://www.reddi...|2023-04-17 20:38:12|Welcome to Micro ...|          32|       pos|       pos|Emotional|
|[FN] Path

In [ ]:

combine_final_df.coalesce(1).write.option("header", "true").csv("final_sentiment_analysis_results.csv", mode="overwrite")


In [ ]:
import pandas as pd

In [ ]:
pandas_test = pd.read_csv("/content/part-00000-c681fe9e-6d8b-40e0-ad60-c6bcfd825425-c000.csv",error_bad_lines=False)

<ipython-input-85-1c35df0ac7c5>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pandas_test = pd.read_csv("/content/part-00000-c681fe9e-6d8b-40e0-ad60-c6bcfd825425-c000.csv",error_bad_lines=False)
Skipping line 315: expected 11 fields, saw 12
Skipping line 349: expected 11 fields, saw 13
Skipping line 365: expected 11 fields, saw 12



In [ ]:
pandas_test.to_csv("Sentiment_Analysis_Report.csv")